In [5]:
import sys
import os
import numpy as np
from tqdm import tqdm
import pickle

# Get the parent folder path (folder K)
parent_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(parent_folder)

# Add folder A to the Python path
folder_classes_path = os.path.join(parent_folder, 'classes')
sys.path.append(folder_classes_path)

/Users/zeyichen/GitHub/Repo/WB_Algo/Stochastic_FP


Load necessary python classes

In [6]:
from true_WB import *
from input_generate_plugin import *
from entropic_estimate_OT import *
from entropic_iterative_scheme import *
from config_log import *
from measure_visualize import *
from input_generate_entropy import *
from sample_plot import *

# Sampler set-up

In [7]:
dim = 2
num_samples = 5000
num_measures = 5
iter = 0
truncated_radius = 150
seed = 1009

In [15]:
source_sampler = MixtureOfGaussians(dim)
source_sampler.random_components(num_components=5, uniform_weights = True, seed = seed) # seed from the measure selection
# source_sampler.set_truncation(truncated_radius)

In [16]:
auxiliary_measure_sampler_set = []
for auxiliary_seed in [1010, 1018, 1014, 1016, 1003]:
    auxiliary_measure_sampler = MixtureOfGaussians(dim)
    auxiliary_measure_sampler.random_components(num_components=5, uniform_weights = True, seed = auxiliary_seed)
    # auxiliary_measure_sampler.set_truncation(truncated_radius)
    auxiliary_measure_sampler_set.append(auxiliary_measure_sampler)

In [17]:
# construct entropic_sampler
entropic_sampler = entropic_input_sampler(dim, 
                                          num_measures, 
                                          auxiliary_measure_sampler_set, 
                                          source_sampler = source_sampler, 
                                          n_k = 1000, 
                                          seed = 120, 
                                          gamma = 0.3, 
                                          manual = True,
                                          truncated_radius = truncated_radius,
                                          bound_type = "eigen_bound")

In [18]:
entropic_sampler.__dict__

{'dim': 2,
 'num_measures': 5,
 'auxiliary_measure_sampler_set': [<true_WB.MixtureOfGaussians at 0x17e653f10>,
 'tilde_K': 5,
 'source_sampler': <true_WB.MixtureOfGaussians at 0x17880b9d0>,
 'n_k': 1000,
 'seed': 120,
 'rng_entropy': RandomState(MT19937) at 0x17E611740,
 'gamma': 0.3,
 'manual': True,
 'truncated_radius': 150,
 'bound_type': 'eigen_bound',
 'grid_size': 200}

# Entropic Sampler Set-up

In [ ]:
# generate strong convexity parameters of the mappings.
entropic_sampler.generate_strong_convexity_param()
print("strong convexity parameters all set.")

In [ ]:
# assign theta
entropic_sampler.assign_theta()
print("theta all set.")

In [ ]:
# generate Y matrices
entropic_sampler.generate_Y_matrices()
print("Y matrices all set.")

In [ ]:
# generate g vectors
entropic_sampler.generate_g_vectors()
print("g vectors all set.")

In [ ]:
# generate smoothness parameters; this involves solving max eigen for each tilde_k
entropic_sampler.generate_smoothness_param()
print("smoothness parameters all set.")

In [ ]:
print(entropic_sampler.smoothness_param_dict)

In [ ]:
# construct a surjective mapping to map component maps to their respective OT maps for generating input measures.
entropic_sampler.construct_surjective_mapping()
print("surjective mapping all set.")

In [ ]:
# generate A matrices
entropic_sampler.generate_A_matrices()
print("A matrices all set.")

In [13]:
epsilon = 10
result_dir = "results"
os.makedirs(result_dir, exist_ok=True)
save_pathname = f"{result_dir}/entropic_measures_{num_measures}_seed_{seed}_samples_{num_samples}_dim_{dim}_epsilon_{epsilon}"
os.makedirs(save_pathname, exist_ok=True)

In [ ]:
import pickle

with open(f"{save_pathname}/entropic_sampler_info.pkl", "wb") as f:
    pickle.dump(entropic_sampler.__dict__, f)

with open(f"{save_pathname}/source_sampler_info.pkl", "wb") as f:
    pickle.dump(source_sampler.__dict__, f)

print(f"Data successfully saved")

# Set-up Over

In [8]:
epsilon = 10
result_dir = "results"
os.makedirs(result_dir, exist_ok=True)
save_pathname = f"{result_dir}/entropic_measures_{num_measures}_seed_{seed}_samples_{num_samples}_dim_{dim}_epsilon_{epsilon}"
os.makedirs(save_pathname, exist_ok=True)

In [9]:
# Load the sampler attributes
with open(f"{save_pathname}/entropic_sampler_info.pkl", "rb") as f:
    loaded_data_entropic_sampler = pickle.load(f)

with open(f"{save_pathname}/source_sampler_info.pkl", "rb") as f:
    loaded_data_source_sampler = pickle.load(f)

print("Loaded Data for Entropic Sampler:", loaded_data_entropic_sampler)
print("Loaded Data for Source Sampler:", loaded_data_source_sampler)

Loaded Data for Entropic Sampler: {'dim': 2, 'num_measures': 5, 'auxiliary_measure_sampler_set': [<true_WB.MixtureOfGaussians object at 0x17dc1b4d0>, <true_WB.MixtureOfGaussians object at 0x17dc0ad50>, <true_WB.MixtureOfGaussians object at 0x17dc0b1d0>, <true_WB.MixtureOfGaussians object at 0x17dc0b650>, <true_WB.MixtureOfGaussians object at 0x17dc0bad0>], 'tilde_K': 5, 'source_sampler': <true_WB.MixtureOfGaussians object at 0x17dc24050>, 'n_k': 1000, 'seed': 120, 'rng_entropy': RandomState(MT19937) at 0x10D75A240, 'gamma': 0.3, 'manual': True, 'truncated_radius': 150, 'bound_type': 'eigen_bound', 'grid_size': 200, 'strong_convexity_param_dict': {0: 0.0001, 1: 0.0001, 2: 0.0001, 3: 0.0001, 4: 0.0001}, 'theta_dict': {0: 10, 1: 10, 2: 10, 3: 10, 4: 10}, 'Y_matrix_dict': {0: array([[ 83.25389778,  -6.77748404],
       [-24.38674772, -23.76266702],
       [ 97.2196175 ,  -0.66185246],
       ...,
       [-35.17059538,  10.05658482],
       [-14.49614928, -84.75519572],
       [-10.84955811

In [10]:
source_sampler = MixtureOfGaussians(dim)
source_sampler.__dict__.update(loaded_data_source_sampler)

entropic_sampler = entropic_input_sampler(dim, num_measures, auxiliary_measure_sampler_set, source_sampler = source_sampler, n_k = 1000, seed = 120)
entropic_sampler.__dict__.update(loaded_data_entropic_sampler)

In [11]:
entropic_sampler.__dict__

{'dim': 2,
 'num_measures': 5,
 'auxiliary_measure_sampler_set': [<true_WB.MixtureOfGaussians at 0x17dc1b4d0>,
 'tilde_K': 5,
 'source_sampler': <true_WB.MixtureOfGaussians at 0x17dc24050>,
 'n_k': 1000,
 'seed': 120,
 'rng_entropy': RandomState(MT19937) at 0x10D75A240,
 'gamma': 0.3,
 'manual': True,
 'truncated_radius': 150,
 'bound_type': 'eigen_bound',
 'grid_size': 200,
 'strong_convexity_param_dict': {0: 0.0001,
  1: 0.0001,
  2: 0.0001,
  3: 0.0001,
  4: 0.0001},
 'theta_dict': {0: 10, 1: 10, 2: 10, 3: 10, 4: 10},
 'Y_matrix_dict': {0: array([[ 83.25389778,  -6.77748404],
         [-24.38674772, -23.76266702],
         [ 97.2196175 ,  -0.66185246],
         ...,
         [-35.17059538,  10.05658482],
         [-14.49614928, -84.75519572],
         [-10.84955811, -17.94279971]]),
  1: array([[-16.97145061,  91.46456815],
         [ 98.16619451, -18.46596147],
         [-75.22131842, -52.44109978],
         ...,
         [ 47.97628268,  31.75763179],
         [-78.94388382, -55.91

In [12]:
# Ensure the plot directory exists
plot_dirc = f"{save_pathname}/plots"
os.makedirs(plot_dirc, exist_ok=True)

In [20]:
visualize_num_samples = 1000

# Sample and plot the source measure
source_measure_samples = source_sampler.sample(visualize_num_samples, multiplication_factor=1)
print("Start to plot the source measure")
plot_2d_source_measures_kde(source_measure_samples, plot_dirc=plot_dirc, scatter=False)
print("Finish plotting the source measure")

# Sample input measures
input_measure_samples = entropic_sampler.sample(visualize_num_samples)

# Set up a single row of subplots for input measures
num_measures = len(input_measure_samples)
fig, axes = plt.subplots(1, num_measures, figsize=(5 * num_measures, 5))

if num_measures == 1:
    axes = [axes]  # Ensure axes is iterable when there's only one subplot

for measure_index, ax in enumerate(axes):
    measure_samples = np.array(input_measure_samples[measure_index])
    print(f"Start to plot the input measure {measure_index}")
    
    # Plot the KDE for the input measure
    plot_2d_input_measure_kde_row(measure_samples, measure_index, scatter=False, ax=ax)
    ax.set_title(r"$\widetilde{{\nu}}_{%d}$" % measure_index)
    print(f"Finish plotting the input measure {measure_index}")

# Save the row plot of input measures
plt.tight_layout()
input_plot_path = os.path.join(plot_dirc, "input_measures_row_plot.png")
plt.savefig(input_plot_path)
plt.close()

print(f"All input measures plotted and saved at {input_plot_path}")


source sampling: 100%|██████████| 1000/1000 [00:00<00:00, 11800.78it/s]


Start to plot the source measure
Finish plotting the source measure


Generating 1000 input measure samples: 100%|██████████| 1000/1000 [00:05<00:00, 167.87it/s]


Start to plot the input measure 0
Finish plotting the input measure 0
Start to plot the input measure 1
Finish plotting the input measure 1
Start to plot the input measure 2
Finish plotting the input measure 2
Start to plot the input measure 3
Finish plotting the input measure 3
Start to plot the input measure 4
Finish plotting the input measure 4
All input measures plotted and saved at results/entropic_measures_5_seed_1009_samples_5000_dim_2_epsilon_10/plots/input_measures_row_plot.png


In [17]:
plot_2d_gmm_pdf(source_sampler, truncated_radius, save_path = plot_dirc, file_name="source_GMM_pdf", title_name = r"$\bar{\nu}$")

In [18]:
i = 0
for auxiliary_measure_sampler in auxiliary_measure_sampler_set:
    plot_2d_gmm_pdf(auxiliary_measure_sampler, truncated_radius, save_path = plot_dirc, file_name=f"auxiliary_{i}_pdf", title_name = r"$\widetilde{{\nu}}_{%d}$" % i)
    i += 1

In [22]:
iterative_computer = entropic_iterative_scheme(dim, 
                             num_measures, 
                             source_sampler, 
                             entropic_sampler, 
                             source_sampler_seed = 1009,
                             truncate_radius = truncated_radius,
                             log = True)

# Iteration starts

In [18]:
source_measure_samples = source_sampler.sample(num_samples, multiplication_factor=1)

source sampling: 100%|██████████| 5000/5000 [00:00<00:00, 13138.67it/s]


In [19]:
input_measure_samples = entropic_sampler.sample(num_samples)

Generating 5000 input measure samples: 100%|██████████| 5000/5000 [00:48<00:00, 103.94it/s]


In [20]:
iterative_computer.V_value_compute(source_measure_samples, input_measure_samples, iter = None, save_pathname = save_pathname)
print(iterative_computer.V_values)

{'true_V_value': 5310.632035802339, 'iteration_0': 8102.337585639351, 'iteration_1': 5398.753367639185}


In [21]:
iterative_computer.converge(source_measure_samples, input_measure_samples, iter, num_samples, max_iter=20, epsilon = epsilon, plot = False, scatter = False)

sampling from the pushforward measure at iteration_0: 100%|██████████| 5000/5000 [00:00<00:00, 10572.73it/s]
100%|██████████| 5/5 [00:20<00:00,  4.03s/it]
sampling from the pushforward measure at iteration_1: 100%|██████████| 5000/5000 [00:45<00:00, 110.47it/s]
100%|██████████| 5/5 [05:32<00:00, 66.50s/it] 
sampling from the pushforward measure at iteration_2: 100%|██████████| 5000/5000 [00:48<00:00, 103.67it/s]
100%|██████████| 5/5 [05:09<00:00, 61.87s/it] 
sampling from the pushforward measure at iteration_3: 100%|██████████| 5000/5000 [01:38<00:00, 50.56it/s]
100%|██████████| 5/5 [05:35<00:00, 67.19s/it] 
sampling from the pushforward measure at iteration_4: 100%|██████████| 5000/5000 [02:27<00:00, 33.91it/s]
100%|██████████| 5/5 [05:28<00:00, 65.78s/it] 
sampling from the pushforward measure at iteration_5: 100%|██████████| 5000/5000 [01:47<00:00, 46.31it/s]
100%|██████████| 5/5 [05:26<00:00, 65.21s/it] 
sampling from the pushforward measure at iteration_6: 100%|██████████| 5000/50

# Plot the mapping trajectory

In [24]:
iterative_computer_forplot = entropic_iterative_scheme(dim, 
                             num_measures, 
                             source_sampler, 
                             entropic_sampler, 
                             source_sampler_seed = 1009,
                             truncate_radius = truncated_radius,
                             log = True)

In [ ]:
iterative_computer.converge(source_measure_samples, input_measure_samples, iter, num_samples, max_iter=20, epsilon = epsilon, plot = False, scatter = False)

# Construct quantiles for V_values of the true barycenter

In [25]:
source_sampler.__dict__

{'truncation': False,
 'weights': array([0.2, 0.2, 0.2, 0.2, 0.2]),
 'gaussians': [(array([-33.28037665,  11.39824015]),
   array([[439.01186754,  38.71218863],
          [ 38.71218863, 335.1910405 ]])),
  (array([ 6.9395913 , 42.45362413]),
   array([[175.55809524,  22.71080606],
          [ 22.71080606, 373.07614477]])),
  (array([ 41.97210095, -43.60649594]),
   array([[257.65936594,  39.64188104],
          [ 39.64188104, 286.95180585]])),
  (array([  9.54330814, -26.98487487]),
   array([[484.51190908, 132.12488889],
          [132.12488889, 311.21569812]])),
  (array([-41.19007905,  32.58518001]),
   array([[ 561.97328857, -129.00902671],
          [-129.00902671,  308.60400296]]))],
 'dim': 2,
 'num_components': 5,
 'seed': 1009}

In [29]:
from tqdm import tqdm, tqdm_notebook

true_V_value_record = []

for i in tqdm(range(200), desc = "Generate source samples for statistical inferences"):
    source_measure_samples = source_sampler.sample(num_samples, multiplication_factor=1)
    input_measure_samples = entropic_sampler.sample(num_samples)
    V_value = 0
    for measure_index in range(num_measures):
        input_samples = np.array(input_measure_samples[measure_index])
        V_value += W2_pot(input_samples, source_measure_samples)
    # normalize the V_value by the number of input measures
    V_value /= num_measures
    true_V_value_record.append(V_value)

true_V_value_record

Generating 5000 input measure samples: 100%|██████████| 5000/5000 [00:15<00:00, 331.73it/s]
Generate source samples for statistical inferences: 100%|██████████| 200/200 [2:03:54<00:00, 37.17s/it]


[5326.25622353957,
 5423.31208094494,
 5462.436865698557,
 5467.682585198126,
 5341.811677353145,
 5405.849645738055,
 5371.635088924129,
 5387.578427364112,
 5455.994896342569,
 5471.187432046214,
 5369.222160000982,
 5441.025876833037,
 5436.003976786344,
 5405.857564114328,
 5354.54416292481,
 5365.334282186077,
 5381.7161654574065,
 5408.734120188639,
 5327.463300656002,
 5355.356950671961,
 5378.136089439499,
 5452.812355973835,
 5459.167169958424,
 5471.888717477226,
 5238.979164987495,
 5428.377775983541,
 5398.649545253207,
 5370.078881100034,
 5369.647834942807,
 5292.042005782798,
 5356.128272569533,
 5494.4552372462795,
 5469.38238112616,
 5300.23060659301,
 5386.270541176387,
 5398.006802437087,
 5423.461996446533,
 5386.642198658686,
 5372.792053212663,
 5468.900543477523,
 5425.655469288531,
 5349.133556263338,
 5316.03049611936,
 5379.835272575296,
 5409.952092557645,
 5415.904905877526,
 5411.429546529078,
 5438.75088584596,
 5528.680930631164,
 5366.063347226547,
 5404

In [5]:
true_V_value_record = np.array([5326.25622353957,
 5423.31208094494,
 5462.436865698557,
 5467.682585198126,
 5341.811677353145,
 5405.849645738055,
 5371.635088924129,
 5387.578427364112,
 5455.994896342569,
 5471.187432046214,
 5369.222160000982,
 5441.025876833037,
 5436.003976786344,
 5405.857564114328,
 5354.54416292481,
 5365.334282186077,
 5381.7161654574065,
 5408.734120188639,
 5327.463300656002,
 5355.356950671961,
 5378.136089439499,
 5452.812355973835,
 5459.167169958424,
 5471.888717477226,
 5238.979164987495,
 5428.377775983541,
 5398.649545253207,
 5370.078881100034,
 5369.647834942807,
 5292.042005782798,
 5356.128272569533,
 5494.4552372462795,
 5469.38238112616,
 5300.23060659301,
 5386.270541176387,
 5398.006802437087,
 5423.461996446533,
 5386.642198658686,
 5372.792053212663,
 5468.900543477523,
 5425.655469288531,
 5349.133556263338,
 5316.03049611936,
 5379.835272575296,
 5409.952092557645,
 5415.904905877526,
 5411.429546529078,
 5438.75088584596,
 5528.680930631164,
 5366.063347226547,
 5404.626717632905,
 5412.026485320612,
 5401.673584132302,
 5384.489060058528,
 5441.466692421943,
 5341.280087775953,
 5443.3430169371095,
 5380.212806246237,
 5314.186237257158,
 5520.393627554388,
 5405.429299337489,
 5425.203444130561,
 5416.853345131327,
 5519.608916078461,
 5504.788654106192,
 5527.776094719351,
 5353.151859951648,
 5326.605051928821,
 5463.08033334842,
 5386.375193870694,
 5366.19579901361,
 5502.633684163268,
 5410.716948306904,
 5422.164648941054,
 5414.660013698993,
 5338.425748755614,
 5347.52097810305,
 5422.624069051961,
 5413.38769837546,
 5383.764874513052,
 5442.399044342585,
 5324.3138960459255,
 5407.279158801262,
 5376.207307756433,
 5510.102966586914,
 5628.913566358445,
 5366.304187041094,
 5421.060604910839,
 5320.522130689556,
 5367.538826173861,
 5514.476362582604,
 5377.772366980918,
 5409.838336265629,
 5425.654021915032,
 5348.204697565888,
 5378.773525433166,
 5374.761486133968,
 5424.21480371669,
 5431.105429366182,
 5363.757585006048,
 5320.631020913708,
 5449.619941673945,
 5460.256194781401,
 5400.435175095122,
 5371.491271845893,
 5281.184115768779,
 5338.945506022948,
 5418.248252260018,
 5425.868977923003,
 5356.0426277807455,
 5396.407430986632,
 5269.750997959088,
 5424.052278066238,
 5368.1210435125,
 5505.36342768686,
 5414.486780692945,
 5420.421022114061,
 5343.428014107711,
 5404.639865546827,
 5378.388284657075,
 5374.537100687934,
 5489.421433640548,
 5382.783933292669,
 5409.254315425269,
 5395.310130418776,
 5413.0975339447405,
 5384.854438890796,
 5412.90326480823,
 5282.444643566865,
 5301.207023492925,
 5320.949464958447,
 5531.112835994997,
 5452.061369665666,
 5474.929918535136,
 5344.1654553487915,
 5441.439541760041,
 5388.698890306203,
 5308.985461110199,
 5489.505314775071,
 5525.025418614313,
 5370.222840311637,
 5383.338836938561,
 5457.664761487613,
 5370.122652976996,
 5466.919973115927,
 5466.9275990388605,
 5466.726268151221,
 5305.418199378584,
 5422.762671566945,
 5303.814122962227,
 5328.717905767393,
 5475.144466491467,
 5327.4322012737875,
 5518.903983394355,
 5451.352185895625,
 5433.764811138641,
 5443.956433167499,
 5424.993768580915,
 5486.896528919622,
 5473.958382656026,
 5430.432993427897,
 5458.108631101219,
 5492.587997174778,
 5328.023284058045,
 5470.239533215673,
 5499.054537894965,
 5402.579442977014,
 5390.846840192533,
 5498.126850569609,
 5346.288071188799,
 5464.120649732395,
 5253.321913499329,
 5488.777793189889,
 5463.830607898862,
 5430.103597651132,
 5436.397359957375,
 5415.529094003052,
 5437.395206843505,
 5422.5589417458505,
 5384.876044093231,
 5442.492162252289,
 5367.03254310621,
 5530.3166767464145,
 5339.08580393308,
 5468.874858488256,
 5291.4444569515745,
 5384.887143094719,
 5307.162996063136,
 5375.907433739154,
 5339.663633743618,
 5347.604303113187,
 5431.931158303411,
 5420.518211580225,
 5364.494807882421,
 5473.837395495022,
 5340.263118792794,
 5463.363951704194,
 5361.724184130546,
 5403.989867393713,
 5364.983867244208])



In [6]:
# find mean and variance of the true V values
# construct 99% two-sided confidence interval
import scipy.stats as stats

mean_true_V = np.mean(true_V_value_record)
std_true_V = np.std(true_V_value_record)
conf_interval = stats.norm.interval(0.95, loc=mean_true_V, scale=std_true_V)

print(f"Mean of true V values: {mean_true_V}")
print(f"Standard deviation of true V values: {std_true_V}")
print(f"95% Confidence interval: {conf_interval}")

Mean of true V values: 5405.533208551414
Standard deviation of true V values: 62.2809319259652
95% Confidence interval: (5283.464825052931, 5527.601592049897)


In [ ]:
# save the true V values as a pickle file
save_pathname = f"results/entropic_measures_5_seed_1009_samples_5000_dim_2_epsilon_10"
with open(f"{save_pathname}/true_V_values.pkl", "wb") as f:
    pickle.dump(true_V_value_record, f)

# Load the true V values
with open(f"{save_pathname}/true_V_values.pkl", "rb") as f:
    loaded_true_V_values = pickle.load(f)
